<a href="https://colab.research.google.com/github/AryanWadhwa05/Stock-Analysis-and-Prediction/blob/main/Bad%20LSTM%20with%20Arima%20streamlit_(DND).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Run this cell to load some data
!pip install streamlit
!pip install pyngrok 
!pip install yfinance
!pip install ipykernel>=5.1.2
!pip install pydeck

import tensorflow as tf
import pandas as pd
import cv2
import plotly.express as px
import numpy as np
import yfinance as yf
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import plotly.express as px
import re
import numpy as np
import datetime as dt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error


     |████████████████████████████████| 9.2 MB 4.8 MB/s 
     |████████████████████████████████| 111 kB 31.0 MB/s 
     |████████████████████████████████| 180 kB 31.5 MB/s 
     |████████████████████████████████| 4.3 MB 30.2 MB/s 
     |████████████████████████████████| 164 kB 38.4 MB/s 
     |████████████████████████████████| 76 kB 2.3 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 126 kB 43.9 MB/s 
     |████████████████████████████████| 792 kB 45.1 MB/s 
     |████████████████████████████████| 374 kB 49.0 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=40e1ac812a3b8341fc980b9f4a469538265071d0cf7667896a48fdabe1b082bf
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

     |████████████████████████████████| 745 kB 5.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=31bd238380b6b9a8d79d6c4c19264afefa2eb17ef05583df42a5b16b38543b37
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 11.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipy

FileNotFoundError: ignored

In [ ]:
%%writefile app.py

# All imports
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import plotly.express as px
import re
import numpy as np
import datetime as dt
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

# Streamlit title
st.title("Stock Market Analysis")

# Streamlit Sidebar
st.sidebar.title("Stocks")

st.sidebar.subheader("Stock Search")

# Pick variables such as stock ticker and what we want to train on (Close, High, Open)
stock_search_test = st.sidebar.text_input('Stock Search')
display_cols = st.sidebar.multiselect('Display Data', ["Close","Adj Close","Low","Open","High","Volume"],default = "High")
graph_type = st.sidebar.selectbox('Graph Type', ["Line","Bar","Vega Lite","AltAir","High","Volume"])

# Pick start and end data for datatset
# start_date = st.sidebar.date_input('Start Date')
end_date = st.sidebar.date_input('End Date')

# Ticker string if multiple tickers
yf.pdr_override()
tickers = ""

tickers = stock_search_test.replace(","," ")

# Download data from yahoo finance
data = yf.download(tickers, end=end_date)

dataframe = data[display_cols]
dataframe.reset_index(level=0, inplace=True)

# Writting dataframe to webapp
st.write(dataframe)

# Plot stock data without ml or nlp
plt.plot(dataframe["Date"],dataframe[display_cols])
plt.ylabel("Dollars")
plt.xlabel("Date")
plt.legend()
plt.draw()

st.pyplot(plt)

# Machine Learning starts
cf=dataframe
pf = cf

if cf.shape[0] > 5000:
  st.write("Daatset too big, please use more recent stock")
  quit()

# ARIMA datatset code
train_ARIMA_data, test_ARIMA_data = cf[0:int(len(cf)*0.7)], cf[int(len(cf)*0.7):]

training_ARIMA_data = train_ARIMA_data['High'].values
test_ARIMA_data = test_ARIMA_data['High'].values

# LSTM dataset code
cf.index=cf['Date']

data=cf.sort_index(ascending=True,axis=0)
new_dataset=pd.DataFrame(index=range(0,len(cf)),columns=['Date',"High"])

for i in range(0,len(data)):
    new_dataset["Date"][i]=data['Date'][i]
    new_dataset[display_cols[0]][i]=data[display_cols[0]][i]

new_dataset["Date"].apply('str')
final_dataset=new_dataset.values

train_data=[]
valid_data=[]

# *********************** splitting the dataset
train_data_number = int((len(final_dataset)*0.7))
train_data=final_dataset[0:train_data_number,:]
valid_data=final_dataset[train_data_number:,:]

new_dataset.index=new_dataset.Date
new_dataset.drop("Date",axis=1,inplace=True)

# ********************** Feature Scaling
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(new_dataset)

x_train_data,y_train_data=[],[]

for i in range(60,len(train_data)):
    x_train_data.append(scaled_data[i-60:i,0])
    y_train_data.append(scaled_data[i,0])
    
x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data) #(len(data),60))

x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1)) #(len(data), 60, 1)

# ARIMA architecture
history = [x for x in training_ARIMA_data]
model_predictions = []
model_fc = []
N_test_observations = len(test_ARIMA_data)
for time_point in range(N_test_observations):
    model = ARIMA(history, order=(4,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    fc, se, conf = model_fit.forecast()
    model_fc.append(fc)
    yhat = output[0]
    model_predictions.append(yhat)
    true_test_value = test_ARIMA_data[time_point]
    history.append(true_test_value)
MSE_error = mean_squared_error(test_ARIMA_data, model_predictions)

# LSTM architecture
lstm_model=Sequential()
lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

inputs_data=new_dataset[len(new_dataset)-len(valid_data)-60:].values
inputs_data=inputs_data.reshape(-1,1)
inputs_data=scaler.transform(inputs_data)

lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=25,batch_size=20,verbose=2)

# Testing
X_test=[]
for i in range(60,inputs_data.shape[0]):
    X_test.append(inputs_data[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
predicted_closing_price=lstm_model.predict(X_test)
predicted_closing_price=scaler.inverse_transform(predicted_closing_price)

train_data=new_dataset[:train_data_number]
valid_data=new_dataset[train_data_number:]

valid_data['Predictions']=predicted_closing_price

# Visuliazation LSTM
plt.plot(train_data["High"])
plt.plot(valid_data[['High',"Predictions"]])
st.pyplot(plt)

# Visualization ARIMA
test_set_range = pf[int(len(pf)*0.7):].index
plt.plot(test_set_range, model_predictions, color='blue', marker='o', linestyle='dashed',label='Predicted Price')
plt.plot(test_set_range, test_ARIMA_data, color='red', label='Actual Price')
plt.title('Prices Prediction')
plt.xlabel('Date')
plt.ylabel('Prices')
# plt.xticks(np.arange(881,1259,50), pf.Date[881:1259:50])
plt.legend()
st.pyplot(plt)
# st.write(model_predictions)
# st.write(model_fc)

LSTM_list = []
LSTM_list = valid_data["Predictions"].tolist()
sum_list = []
sum = 0

for i in range(len(model_fc)-1):
  sum = 0
  sum = (model_fc[i] + LSTM_list[1])/2
  sum_list.append(sum)

# Display LSTM prediction datatset
st.write(valid_data)
st.write(sum_list)


Writing app.py


In [ ]:
!ngrok authtoken 23D1NySpe3UL1BoP2zl0GzEhkIw_7ZZp4VJS5pZi9snaZSM2K

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
#Publish Web App (Run this again whenever you make changes)
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py

2022-01-07 03:52:45.989 INFO    pyngrok.ngrok: Opening tunnel named: http-80-3bc7790e-38ae-46f0-9aa4-08e499b18afd
2022-01-07 03:52:46.018 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:46+0000 lvl=info msg="no configuration paths supplied"
2022-01-07 03:52:46.027 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:46+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-01-07 03:52:46.029 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:46+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-01-07 03:52:46.034 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:46+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2022-01-07 03:52:46.154 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:46+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2022-01-07 03:52:46.155 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:46+0000 lvl=info msg="client session established" obj=csess id=5f41cbafa738
2

NgrokTunnel: "http://0610-35-192-38-179.ngrok.io" -> "http://localhost:80"


2022-01-07 03:52:48.516 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:48+0000 lvl=info msg="join connections" obj=join id=48027218204d l=127.0.0.1:80 r=24.188.197.152:56552
2022-01-07 03:52:48.640 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:48+0000 lvl=info msg="join connections" obj=join id=2d5742689f29 l=127.0.0.1:80 r=24.188.197.152:56551
2022-01-07 03:52:48.680 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:48+0000 lvl=info msg="join connections" obj=join id=fc0116722109 l=127.0.0.1:80 r=24.188.197.152:56553
2022-01-07 03:52:48.683 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:48+0000 lvl=info msg="join connections" obj=join id=279e3887a473 l=127.0.0.1:80 r=24.188.197.152:56554
2022-01-07 03:52:48.689 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:48+0000 lvl=info msg="join connections" obj=join id=82de456ea6ea l=127.0.0.1:80 r=24.188.197.152:56555
2022-01-07 03:52:48.694 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:48+0000 lvl=info msg="join connections" o


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://35.192.38.179:80



2022-01-07 03:52:49.497 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:49+0000 lvl=info msg="join connections" obj=join id=c6a3ca556d38 l=127.0.0.1:80 r=24.116.24.38:57892
2022-01-07 03:52:49.848 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:49+0000 lvl=info msg="join connections" obj=join id=abc7949e3f76 l=127.0.0.1:80 r=24.116.24.38:45880
2022-01-07 03:52:49.851 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:49+0000 lvl=info msg="join connections" obj=join id=d1607358a566 l=127.0.0.1:80 r=24.116.24.38:33596
2022-01-07 03:52:49.863 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:49+0000 lvl=info msg="join connections" obj=join id=0c95dff8baf1 l=127.0.0.1:80 r=24.116.24.38:45878
2022-01-07 03:52:49.871 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:49+0000 lvl=info msg="join connections" obj=join id=ef800722a014 l=127.0.0.1:80 r=24.116.24.38:44446
2022-01-07 03:52:50.215 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:50+0000 lvl=info msg="join connections" obj=join id

...

[*********************100%***********************]  0 of 0 completed
2022-01-07 03:52:52.799 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 142, in download
    keys=shared._DFS.keys())
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py", line 284, in concat
    sort=sort,
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py", line 331, in __init__
    raise ValueError("No objects to concatenate")
ValueError: No objects to concatenate

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 47, in <module>
    data = yf.download(tickers, end=end_date)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 146, in download
    keys=shared._DFS.keys())
  

2022-01-07 03:52:53.455 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:53+0000 lvl=info msg="join connections" obj=join id=d39970bfa178 l=127.0.0.1:80 r=24.188.197.152:56556
2022-01-07 03:52:53.460 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:53+0000 lvl=info msg="join connections" obj=join id=087294901e76 l=127.0.0.1:80 r=24.188.197.152:56552
2022-01-07 03:52:53.466 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:53+0000 lvl=info msg="join connections" obj=join id=fe2ec5bf8243 l=127.0.0.1:80 r=24.188.197.152:56551
2022-01-07 03:52:53.531 INFO    pyngrok.process.ngrok: t=2022-01-07T03:52:53+0000 lvl=info msg="join connections" obj=join id=2992de121415 l=127.0.0.1:80 r=24.188.197.152:56552


/content/app.py:19: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

2022-01-07 03:52:56.705 NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
2022-01-07 03:52:56.813 No handles with labels found to put in legend.
2022-01-07 03:54:16.131781: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names

Epoch 1/25
82/82 - 9s - loss: 0.0032 - 9s/epoch - 104ms/step
Epoch 2/25
82/82 - 2s - loss: 1.8021e-04 - 2s/epoch - 25ms/step
Epoch 3/25
82/82 - 2s - loss: 1.6528e-04 - 2s/epoch - 25ms/step
Epoch 4/25
82/82 - 2s - loss: 1.7023e-04 - 2s/epoch - 25ms/step
Epoch 

2022-01-07 04:17:01.996 INFO    pyngrok.process.ngrok: t=2022-01-07T04:17:01+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
2022-01-07 04:17:02.005 INFO    pyngrok.process.ngrok: t=2022-01-07T04:17:01+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


  Stopping...
^C
